<a href="https://colab.research.google.com/github/djolobolonjez/PSIML8-Qualification-tasks/blob/master/Task%202/PSIML8_Roomba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from zipfile import ZipFile
file_name = "/roomba-public.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

'from zipfile import ZipFile\nfile_name = "/roomba-public.zip"\n\nwith ZipFile(file_name, \'r\') as zip:\n  zip.extractall()\n  print(\'Done\')'

Importing libraries

In [ ]:
import numpy as np
from PIL import Image, ImageFilter, ImageOps, ImageDraw, ImageColor
from itertools import product
import math
import matplotlib.pyplot as plt
import os

Gaussian Kernel

In [ ]:
def gaussian_kernel(size = 5, sigma = 1):
  size = int(size)//2
  x, y = np.mgrid[-size:size+1, -size:size+1]
  normal = 1/(2.0*np.pi*sigma**2)
  g = np.exp(-((x**2+y**2)/(2.0*sigma**2)))*normal
  return g

Sobel filter

In [ ]:
def sobel_filter(img):

  k_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], np.float32)
  k_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], np.float32)

  img_x = convolve2D(img, k_x)
  img_y = convolve2D(img, k_y)

  g = np.hypot(img_x, img_y)
  g = g/g.max()*255

  theta = np.arctan2(img_x, img_y)

  return (g, theta)

Double threshold for pixel classification

In [ ]:
def threshold(img, low_threshold_ratio = 0.05, high_threshold_ratio = 0.09):

  high_threshold = img.max()*high_threshold_ratio
  low_threshold = high_threshold*low_threshold_ratio

  m, n = img.shape

  result = np.zeros((m, n))

  weak = 25
  strong = 255

  strong_i, strong_j = np.where(img >= high_threshold)
  zero_i, zero_j = np.where(img <= low_threshold)

  weak_i, weak_j = np.where((img <= high_threshold) & (img >= low_threshold))

  for i in range(len(strong_i)):
    result[strong_i[i]][strong_j[i]] = strong
  
  for i in range(len(weak_i)):
    result[weak_i[i]][weak_j[i]] = weak

  return (result, weak, strong)

Hysteresis for adjacent pixels

In [ ]:
def hysteresis(img, weak_pixel = 75, strong_pixel = 255):

  m, n = img.shape
  weak = weak_pixel
  strong = strong_pixel

  for i in range(1, m-1):
    for j in range(1, n-1):
      if img[i][j] == weak:
        try:
          if((img[i+1][j-1] == strong) or (img[i+1][j] == strong) or (img[i+1][j+1] == strong)
            or (img[i][j-1] == strong) or (img[i][j+1] == strong)
            or (img[i-1][j-1] == strong) or (img[i-1][j] == strong) or img[i-1][j+1] == strong):
            img[i][j] = strong
          else:
            img[i][j] = 0
          
        except IndexError as e:
          pass

  return img


2D convolution for image filtering

In [ ]:
def convolve2D(image, kernel, padding = 0, strides = 1):

  kernel = np.flipud(np.fliplr(kernel))
  x_kern = kernel.shape[0]
  y_kern = kernel.shape[1]
  x_img = image.shape[0]
  y_img = image.shape[1]

  x_out = int(((x_img - x_kern + 2*padding)/strides) + 1)
  y_out = int(((y_img - y_kern + 2*padding)/strides) + 1)

  output = np.zeros((x_out, y_out))

  if padding != 0:
    padded_image = np.zeros((image.shape[0] + 2*padding, image.shape[1] + 2*padding))
    padded_image[int(padding):int(-1*padding), int(padding):int(-1*padding)] = image
  
  else:
    padded_image = image

  for y in range(image.shape[1]):
    if y > image.shape[1] - y_kern:
      break
    
    if y % strides == 0:
      for x in range(image.shape[0]):
        if x > image.shape[0] - x_kern:
          break
        try:
          if x % strides == 0:
            output[x][y] = (kernel*padded_image[x:x + x_kern, y:y + y_kern]).sum()
        except:
          break

  return output

Filling room matrix

In [ ]:
def room_map(grid, image, d, img_h, img_w):
  room_h = 0
  room_w = 0
  total = 0
  img_w, img_h = image.size

  legend = {'empty': 0, 'wall': 1, 'roomba': 2, 'station': 3, 'furniture': 4, 'dirt': 5}

  grid = product(range(0, img_h - img_h%d, d), range(0, img_w-img_w%d, d))
  for i, j in grid:
    total += 1
    if j == 0:
      room_h += 1
  
  room_w = total//room_h

  surface = img_h*img_w

  offset = int(math.sqrt(surface/(room_w*room_h)))

  room_matrix = np.zeros((room_h, room_w), dtype = int)
  h = w = 0
  grid = product(range(0, img_h-img_h%offset, offset), range(0, img_w-img_w%offset, offset))

  for i, j in grid:
    box = (j, i, j+offset, i+offset)
    curr_image = image.crop(box)

    curr_image = np.asarray(curr_image)
    curr_h, curr_w = curr_image.shape[:2]

    middle = curr_image[curr_h//2][curr_w//2]

    if np.sum(middle) == 0:
      obj = "wall"
      
    elif middle[0] == 255 and middle[1] == 255 and middle[2] == 255:
      obj = "empty"

    elif np.argmax(middle) == 1:
      obj = "station"
    
    else:
      if middle[1] == middle[2]:
        obj = "roomba"
      elif middle[0] == 255 and middle[1] > 200:
        obj = "dirt"
      else:
        obj = "furniture"

    room_matrix[h][w] = legend[obj]
    w += 1

    if w == room_w:
      w = 0
      h += 1
  
  return room_matrix, room_h, room_w


Preprocessing

In [ ]:
def preprocessing(imagepath, classification = True):
  image = Image.open(imagepath)
  r, g, b, a = image.split()
  rgb_image = Image.merge('RGB', (r, g, b))

  invert_image = ImageOps.invert(rgb_image)
  bbox = invert_image.getbbox()
  image = image.crop(bbox)
  image = image.convert('RGB')
  edge_image = image.convert("L")

  edge_img = np.asarray(edge_image)
  edge_img = convolve2D(edge_img, gaussian_kernel(3))
  gradient, theta = sobel_filter(edge_img)
  threshold_img, weak, strong = threshold(gradient)
  final_img = hysteresis(threshold_img, weak, strong)

  to_save = Image.fromarray(final_img.astype(np.uint8))
  to_save.save(r"final.png")

  point = [0, 0]
  found = False
  for i in range(final_img.shape[0]):
    for j in range(final_img.shape[1]):
      if (final_img[i][j] == 255):
        point = [i, j]
        found = True
        break
      
    if found: break

  d = point[-1]

  h, w = final_img.shape
  grid = product(range(0, h-h%d, d), range(0, w-w%d, d))
  
  if not classification:
    return room_map(grid, image, d, h, w)


In [ ]:
class Cell:
  def __init__(self, row, col, path):
    self.row = row
    self.col = col
    self.path += path

In [ ]:
def valid(x, y, room_matrix, visited):
  if (x >= 0 and y >= 0) and (x < len(room_matrix) and y < len(room_matrix[0])) and (room_matrix[x][y] != 4) and (room_matrix[x][y] != 1) and (visited[x][y] == False):
    return True
  
  return False

BFS for finding a path to the charging station

In [ ]:
def pathfinding(room_matrix):
  moves = {}
  source_row = source_col = 0

  for row in range(len(room_matrix)):
    for col in range(len(room_matrix[row])):
      if room_matrix[row][col] == 2:
        source_row = row
        source_col = col
        break
  
  source = (source_row, source_col)
  moves[source] = ''
  visited = [[False for _ in range(len(room_matrix[0]))] for _ in range(len(room_matrix))]

  queue = []
  queue.append(source)
  visited[source_row][source_col] = True
  while len(queue):
    row, col = queue.pop(0)
    
    if room_matrix[row][col] == 3:
      return moves[(row, col)]

    if valid(row - 1, col, room_matrix, visited):
      cell = (row - 1, col)
      moves[cell] = moves[(row, col)] + 'u'
      queue.append(cell)
      visited[row - 1][col] = True

    if valid(row + 1, col, room_matrix, visited):
      cell = (row + 1, col)
      moves[cell] = moves[(row, col)] + 'd'
      queue.append(cell)
      visited[row + 1][col] = True

    if valid(row, col - 1, room_matrix, visited):
      cell = (row, col - 1)
      moves[cell] = moves[(row, col)] + 'l'
      queue.append(cell)
      visited[row][col - 1] = True

    if valid(row, col + 1, room_matrix, visited):
      cell = (row, col + 1)
      moves[cell] = moves[(row, col)] + 'r'
      queue.append(cell)
      visited[row][col + 1] = True    


Main

In [ ]:
imagepath, C, Cmax = input().split(' ')
C = int(C)
Cmax = int(Cmax)

shapes = {}

room_matrix, H, W = preprocessing(imagepath, False)

print([C, Cmax])
print('Task 1')
print([H, W])
print(*room_matrix, sep = "\n")

path = pathfinding(room_matrix)
task2 = []
for char in path:
  task2.append(char)

print('Task 2')
print(task2)


/content/public/set/room_3.png 2 2
[2, 2]
Task 1
[6, 6]
[1 1 1 1 1 1]
[1 3 4 5 0 1]
[1 0 0 0 0 1]
[1 5 4 4 0 1]
[1 4 5 2 0 1]
[1 1 1 1 1 1]
Task 2
['r', 'u', 'u', 'l', 'l', 'l', 'u']
